In [341]:
import numpy as np
from scipy.stats import norm
d = 2 #1000 #dimension of the input space 
YF = 9 #200 #critical threshold (gx>=YF)
pF = 1 - norm.cdf(YF/np.sqrt(d)) #true value of the failure proability
n = 1000 #3000 #number of samples per level
p = 0.1 #level probability
nc = int(n*p) #number of Markov chains
ns = int((1-p)/p) #number of states in each chain

In [342]:
L = 0 # zero level
Lmax = 100 # define the possible maximum of number of levels
X = np.zeros((n,d,Lmax)) #samples for different levels
x = np.random.randn(n,d) # Monte Carlo samples(initial samples)
X[:,:,0] = x ### samples for zero level
nF = np.zeros(Lmax) # number of failure samples
y = np.zeros((Lmax,n)) # responses
z = np.zeros((nc,d,ns+1)) # seeds
Y = np.zeros(Lmax)
q = np.zeros(d)## potential next state in Markov chain?


In [344]:
for i in range(1, n+1):
    y[0,i-1] = sum(x[i-1,:]) #zero level response
    if y[0,i-1]>YF: # whether it falls in failure domain
        nF[0]=nF[0]+1

In [345]:
while (nF[L]/n)<p: # stopping criterion
    L=L+1 # next conditional level is needed
    w = -y[L-1,:]
    ind = sorted(range(len(w)), key=lambda k: w[k])# original index of descending responses
    y[L-1,:] = y[L-1,ind] #renumbered responses
    X[:,:, L-1] = X[ind,:,L-1] # renumbered samples
    Y[L-1] = (y[L-1,nc-1]+y[L-1,nc])/2 #L^th intermediate threshold
    z[:,:,0] = X[range(0,nc),:, L-1] # seeds of Markov chain
    ###modified metropolis
    for j in range(1,nc+1):
        for m in range(1,ns+1):
            ### step 1
            for k in range(1,d+1):
                a = z[j-1,k-1,m-1]+np.random.randn(1)
                r = min(1, norm.pdf(a)/norm.pdf(z[j-1,k-1,m-1]))
                if np.random.rand(1)<r:
                    q[k-1]=a
                else:
                    q[k-1]=z[j-1,k-1,m-1]
            ###step 2 q belongs to F_L or not
            if sum(q)>Y[L-1]:
                z[j-1,:,m]=q
            else:
                z[j-1,:,m]=z[j-1,:,m-1]
    for j in range(1,nc+1):
        for m in range(1,ns+2):
            X[(j-1)*(ns+1)+m-1,:,L]=z[j-1,:,m-1]
    z = np.zeros((nc,d,ns+1))
    nF[L] = 0
    for i in range(1,n+1):
        y[L, i-1] = sum(X[i-1,:,L]) ##responses
        if y[L,i-1]>YF:
            nF[L]=nF[L]+1


In [346]:
pF_SS = (p**(L))*(nF[L])/n
N=n+n*(1-p)*(L)